In [ ]:
# Import necessary libraries.
import os
import glob
import imageio
import random, shutil
import torch
import torch.nn as nn
from tqdm.notebook import tqdm
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as display
import librosa
import librosa.display

In [ ]:
import requests

fname = "music.zip"
url = "https://osf.io/drjhb/download"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)

In [ ]:
from zipfile import ZipFile

with ZipFile(fname, 'r') as zipObj:
  # Extract all the contents of zip file in different directory
  zipObj.extractall()

In [ ]:
import pandas as pd
import shutil

In [ ]:
dir= 'Data/genres_original/' # the dir with all the genres
new_dir= 'new_music' # new dir for us

shutil.rmtree(new_dir, ignore_errors = False)
os.mkdir(new_dir)
genre_dict={}
for genre in os.listdir( dir ):
    genre_dir= dir+genre 
  
    os.mkdir(f'{new_dir}/{genre}')

    genre_dict[genre]= []
    print(f'we are in {genre} directory')
    use_this_dir= f'{new_dir}/{genre}' # move files here
    for music_file in os.listdir(genre_dir): # this is the music file name
        music_loc =os.path.join( genre_dir ,music_file) # the address for the music file
        
        try:
            y, sr = librosa.load(music_loc)
            y_resamble= librosa.resample(y, 22050, 11025) # using subsampling

            S = librosa.feature.melspectrogram( y_resamble, sr=11025 )
            S_DB = librosa.amplitude_to_db(S, ref=np.max)

            genre_dict[genre].append( S_DB )  # tries to add the signal to the genre

        except:
            print('error')

In [ ]:
# taking only 99 signals instead of 100 because we have 1 faulty file inside genre: jazz
for arr in genre_dict.keys():
    if arr == 'jazz':
        pass
    else:
        del genre_dict[arr][-1]

In [ ]:
df= pd.DataFrame( genre_dict)# , dtype= np.float32 )

In [ ]:
df_as_array= df.to_numpy()

In [ ]:
np.save('data_array.npy' , df_as_array  )

In [ ]:
data_tmp = np.load('/content/data_array.npy' , allow_pickle=True)

In [ ]:
data= pd.DataFrame(data_tmp)

In [ ]:
# creating X for the signals
# y for the labels
y=[]
x=[]
for label in data:
    for sig in data[label]:
        y.append(label)
        x.append(sig)


In [ ]:
tmp_arr=np.zeros((990,1,128,645))
for i in range(tmp_df.shape[0]):
    tmp_arr[i]=tmp_df.iloc[i][0][:,:645]

In [ ]:
X= np.squeeze( tmp_arr.copy())

In [ ]:
y=tmp_df.index.to_numpy()

## saving the features and labels

In [ ]:
np.save( 'features.npy',X)
np.save('labels.npy' ,y)

# loading the data

In [ ]:
x_load= np.load('features.npy')
y_load= np.load('labels.npy')

In [ ]:
from  sklearn.model_selection import train_test_split 
x_train , x_test , y_train , y_test = train_test_split( x_load , y_load , train_size=0.8)

# changing them into torch format
x_train = torch.tensor(x_train )
x_test = torch.tensor(x_test)

y_train =torch.tensor(y_train).type(torch.IntTensor) 
y_test = torch.tensor(y_test).type(torch.IntTensor)

train = torch.utils.data.TensorDataset(x_train,y_train)
test = torch.utils.data.TensorDataset(x_test,y_test)